In [ ]:
import pandas as pd
from datetime import datetime
from googleapiclient.discovery import build

# Proceso y Metodología de Extracción de Comentarios

Se busca extraer los links de los videos de los canales seleccionados en el dataset de **noticias** hasta que estos sumen, como mínimo, 250,000 comentarios en su conjunto, para aquellos que tienen una categoría distinta a "Internacional".

## Criterio de Selección

- **Palabras Claves**: Las palabras claves estarán contenidas en el dataset 'claves' (nombre del presidente, país del presidente) junto con la palabra "presidente".
- **Número mínimo de comentarios:** El minimo de número de comentarios del video son 100 para que este sea considerado en la lista.
- **Dataset 'claves'**: Este dataset tiene 2 columnas: una llamada "País" y otra "Nombre del Presidente". Para esta búsqueda solo se usará la columna "Nombre del Presidente".
- **Periodo de Tiempo**: Los videos deben ser del último año.
- **Filtrado Inicial**: Inicialmente, solo se buscará filtrar los videos que contengan los criterios de las palabras y conocer cuántos comentarios tienen.

In [65]:
noticias=  pd.read_excel('canales_noticias_con_id.xlsx')

In [66]:
noticias.head()

,Nombre del noticiero,Categoría,Link,ChannelID
0,France 24 Español,Internacional,https://www.youtube.com/@France24_es,UCUdOoVWuWmgo1wByzcsyKDQ
1,Euronews en Español,Internacional,https://www.youtube.com/@euronewses,UCyoGb3SMlTlB8CLGVH4c8Rw
2,CNN en Español,Internacional,https://www.youtube.com/@cnnee,UCJag7ggH_TnSye399htVAEw
3,DW Español,Internacional,https://www.youtube.com/@dwespanol,UChmI8e6G2mV-0zf9SuQdbVA
4,Telemundo,Internacional,https://www.youtube.com/@noticias,UC2Xq2PK-got3Rtz9ZJ32hLQ


In [14]:
claves = pd.read_excel("data/tables/presidentes2024.xlsx")

In [15]:
claves.head()

,País,Nombre del Presidente
0,Argentina,Javier Milei
1,El Salvador,Nayib Bukele
2,Colombia,Gustavo Petro
3,México,Andrés Manuel López Obrador


In [48]:
api_key = "your_api_key"

In [49]:
youtube = build('youtube', 'v3', developerKey=api_key)

### Descripción de la Función `get_video_details`

#### Propósito
Obtiene detalles de un video en YouTube utilizando la API de YouTube Data.

#### Parámetro
- `video_id`: El identificador único del video en YouTube.

#### Funcionamiento
1. Llama a la API de YouTube para obtener datos del video (`snippet`, `contentDetails`, `statistics`).
2. Verifica si la respuesta contiene elementos y selecciona el primer video.
3. Extrae y almacena los siguientes detalles:
   - Título del video
   - Descripción
   - Fecha de publicación
   - Duración
   - Número de comentarios
4. Crea un diccionario con estos detalles y lo devuelve. Si no hay datos, devuelve `None`.

In [70]:
def get_video_details(video_id):
    video_response = youtube.videos().list(
        part="snippet,contentDetails,statistics", # se recolecta la información de los metadatos del video y las estadísticas
        id=video_id
    ).execute()
    
    if "items" in video_response and len(video_response["items"]) > 0: # si se encontró el video en youtube se extrae la información
        video = video_response["items"][0] # se extrae la información del video en cuestión 
        title = video["snippet"]["title"] # se extrae el título del video
        description = video["snippet"]["description"] # se extrae la descripción del video
        published_at = video["snippet"]["publishedAt"] # se extrae la fecha de publicación del video
        duration = video["contentDetails"]["duration"] # se extrae la duración del video
        comment_count = int(video["statistics"].get("commentCount", 0)) # se extrae el número de comentarios del video 
        video_data = { # se crea un diccionario con la información del video
            "Nombre del Video": title,
            "Link del Video": f"https://www.youtube.com/watch?v={video_id}",
            "Numero de Comentarios": comment_count,
            "Duración del Video": duration,
            "Fecha del Video": published_at
        }
        return video_data
    return None

In [71]:
# Concatenar las columnas 'País' y 'Nombre del Presidente' de 'claves' y añadir la palabra "presidente"
claves['Query'] = claves['País'] + " " + claves['Nombre del Presidente'] + " presidente"

## Extracción de datos de enero

In [45]:
# Obtener la fecha actual y establecer las fechas de inicio y fin para el mes de enero del año actual
current_date = datetime.now()
january_first = datetime(current_date.year, 1, 1).isoformat("T") + "Z"
january_end = datetime(current_date.year, 1, 31, 23, 59, 59).isoformat("T") + "Z"

### Función para buscar videos en un canal con palabras clave específicas y dentro del rango de fechas especificado

#### Propósito
Busca videos en un canal de YouTube que coincidan con palabras clave específicas y un rango de fechas determinado.

#### Parámetros
- `channel_id`: El identificador único del canal de YouTube.
- `query`: Las palabras clave para buscar videos.
- `published_after`: La fecha de inicio del rango (en formato ISO 8601).
- `published_before`: La fecha de fin del rango (en formato ISO 8601).
- `max_results` (opcional): El número máximo de resultados a devolver (por defecto, 5).

#### Funcionamiento
1. **Llamada a la API de Búsqueda de YouTube**:
   - Realiza una búsqueda en el canal especificado utilizando las palabras clave y el rango de fechas.
   - Ordena los resultados por cantidad de visualizaciones y limita el número de resultados según `max_results`.

2. **Recopilación de Resultados**:
   - Itera sobre los resultados de búsqueda.
   - Para cada video encontrado, obtiene detalles específicos utilizando la función `get_video_details`.

3. **Filtrado y Almacenamiento de Videos**:
   - Filtra los videos que tienen más de 100 comentarios.
   - Almacena los datos de los videos que cumplen con este criterio en una lista.

4. **Devolución de Resultados**:
   - Devuelve la lista de videos que cumplen con los criterios de búsqueda y tienen más de 100 comentarios.

In [46]:
# Función para buscar videos en un canal con palabras clave específicas y dentro del rango de fechas especificado
def search_videos(channel_id, query, published_after, published_before, max_results=5):
    search_response = youtube.search().list(
        channelId=channel_id, # se busca en el canal con el ID proporcionado
        q=query, # se busca la palabra clave en los títulos y descripciones
        type="video", # se buscan solo videos
        part="id,snippet", # se recolectan los metadatos del video y la información de la búsqueda
        maxResults=max_results, # se obtienen 5 resultados como máximo por búsqueda dado el límite de cuotas
        order='viewCount', # se ordenan los resultados por vistas
        publishedAfter=published_after, # se obtienen solo los videos publicados después de la fecha especificada
        publishedBefore=published_before   # se obtienen solo los videos publicados antes de la fecha especificada
    ).execute()
    
    videos = []
    for search_result in search_response.get("items", []): # se recorren los resultados de la búsqueda
        video_id = search_result["id"]["videoId"] # se extrae el ID del video
        video_data = get_video_details(video_id) # se obtiene la información detallada del video
        if video_data and video_data["Numero de Comentarios"] > 100: # si se encontró información y el video tiene más de 100 comentarios
            videos.append(video_data) # se añade la información del video a la lista de videos
    return videos


In [130]:
noticias

,Nombre del noticiero,Categoría,Link,ChannelID
0,France 24 Español,Internacional,https://www.youtube.com/@France24_es,UCUdOoVWuWmgo1wByzcsyKDQ
1,Euronews en Español,Internacional,https://www.youtube.com/@euronewses,UCyoGb3SMlTlB8CLGVH4c8Rw
2,CNN en Español,Internacional,https://www.youtube.com/@cnnee,UCJag7ggH_TnSye399htVAEw
3,DW Español,Internacional,https://www.youtube.com/@dwespanol,UChmI8e6G2mV-0zf9SuQdbVA
4,Telemundo,Internacional,https://www.youtube.com/@noticias,UC2Xq2PK-got3Rtz9ZJ32hLQ
5,TN - Todo Noticias,Argentina,https://www.youtube.com/@todonoticias,UCyS7HkaIUX2FBJkJUsnFZUA
6,C5N,Argentina,https://www.youtube.com/@c5n,UCFgk2Q2mVO1BklRQhSv6p0w
7,Canal 26,Argentina,https://www.youtube.com/@canal26,UCrpMfcQNog595v5gAS-oUsQ
8,La Nación,Argentina,https://www.youtube.com/@lanacion,UCba3hpU7EFBSk817y9qZkiA
9,A24,Argentina,https://www.youtube.com/@A24com,UCR9120YBAqMfntqgRTKmkjQ


In [ ]:
claves

### Función para buscar videos en múltiples canales y guardarlos en un archivo CSV

#### Propósito
Busca videos en múltiples canales de YouTube utilizando palabras clave específicas dentro de un rango de fechas y guarda los resultados en un archivo CSV.

#### Parámetros
- `noticias`: DataFrame que contiene información sobre los noticieros, incluidas las categorías, nombres y IDs de los canales.
- `claves`: DataFrame que contiene las palabras clave para la búsqueda, incluyendo los nombres de los presidentes y sus países.
- `january_first`: Fecha de inicio del rango (1 de enero en formato ISO 8601).
- `january_end`: Fecha de fin del rango (31 de enero en formato ISO 8601).

#### Funcionamiento
1. **Inicialización**:
   - Crea una lista vacía `videos` para almacenar los datos de los videos encontrados.

2. **Iteración sobre los canales**:
   - Recorre cada fila del DataFrame `noticias`.
   - Extrae el `ChannelID`, `Categoría` y `Nombre del noticiero`.

3. **Verificación y Búsqueda de Videos**:
   - Si el `ChannelID` no es nulo, itera sobre cada fila del DataFrame `claves`.
   - Extrae la `Query` (consulta de búsqueda) y el `País`.
   - Llama a la función `search_videos` con el `channel_id`, `query`, `january_first` y `january_end`.

4. **Procesamiento de Resultados de Búsqueda**:
   - Para cada resultado de búsqueda, si el video tiene más de 100 comentarios, se agregan datos adicionales:
     - `Nombre del noticiero`
     - `Query` utilizada
     - `País` (se ajusta dependiendo de si la categoría es "Internacional")

5. **Almacenamiento de Resultados**:
   - Los detalles de los videos encontrados se agregan a la lista `videos`.

6. **Guardado en CSV**:
   - Se convierte la lista `videos` en un DataFrame `df_videos`.
   - Se guarda el DataFrame en un archivo CSV (`./videos-per-month/enero.csv`).

7. **Actualización de `claves`**:
   - Se concatena `País` y `Nombre del Presidente` en `claves` para formar la `Query`.

In [51]:
# Buscar videos y guardar en CSV
videos = []
for _, row in noticias.iterrows(): # Recorrer cada fila de 'noticias'
    channel_id = row['ChannelID'] # Obtener el ID del canal de la fila actual
    categoria = row['Categoría'] # Obtener la categoría del noticiero
    nombre_noticiero = row['Nombre del noticiero'] # Obtener el nombre del noticiero
    
    if pd.notnull(channel_id): # Si el ID del canal no es nulo
        for _, clave_row in claves.iterrows(): # Recorrer cada fila de 'claves'
            query = clave_row['Query'] # Obtener la consulta de la fila actual
            country = clave_row['País'] # Obtener el país de la fila actual
            search_results = search_videos(channel_id, query, january_first, january_end) # Buscar videos en el canal con la consulta y fechas especificadas
            for video in search_results: # Recorrer cada video encontrado en los resultados de la búsqueda
                if video:
                    video["Nombre del noticiero"] = nombre_noticiero # Agregar el nombre del noticiero a los datos del video
                    video["Query"] = query  # Agregar la consulta a los datos del video
                    if categoria == "Internacional": # Si la categoría del noticiero es internacional
                        video["País"] = f"Internacional + {country}"
                    else:
                        video["País"] = categoria + " " + country # Agregar el país a los datos del video
                    videos.append(video)

# Crear DataFrame y guardar en CSV
df_videos = pd.DataFrame(videos)
df_videos.to_csv("./videos-per-month/enero.csv", index=False)

print("Videos data saved to ./videos-per-month/enero.csv")

Videos data saved to ./videos-per-month/enero.csv


In [52]:
df_videos["Numero de Comentarios"].sum()

74888

## Extracción de datos de febrero

In [99]:
api_key = "your_api_key"

In [100]:
youtube = build('youtube', 'v3', developerKey=api_key)

In [94]:
current_date = datetime.now()
february_first = datetime(current_date.year, 2, 1).isoformat("T") + "Z"
february_end = datetime(current_date.year, 2, 29, 23, 59, 59).isoformat("T") + "Z"

Se vuelve a correr la consulta de el mes de enero.

In [101]:
# Buscar videos y guardar en CSV
videos = []
for _, row in noticias.iterrows():
    channel_id = row['ChannelID']
    categoria = row['Categoría']
    nombre_noticiero = row['Nombre del noticiero']
    
    if pd.notnull(channel_id):
        for _, clave_row in claves.iterrows():
            query = clave_row['Query']
            country = clave_row['País']
            search_results = search_videos(channel_id, query, february_first, february_end)
            for video in search_results:
                if video:
                    video["Nombre del noticiero"] = nombre_noticiero
                    video["Query"] = query  # Agregar la consulta a los datos del video
                    if categoria == "Internacional":
                        video["País"] = f"Internacional + {country}"
                    else:
                        video["País"] = categoria
                    videos.append(video)

# Crear DataFrame y guardar en CSV
df_videos = pd.DataFrame(videos)
df_videos.to_csv("./videos-per-month/febrero.csv", index=False)

print("Videos data saved to ./videos-per-month/febrero.csv")

Videos data saved to ./videos-per-month/febrero.csv


In [102]:
df_videos["Numero de Comentarios"].sum()

152653

## Extracción de datos de Marzo

In [103]:
api_key = "your_api_key"

In [104]:
youtube = build('youtube', 'v3', developerKey=api_key)

In [105]:
current_date = datetime.now()
march_first = datetime(current_date.year, 3, 1).isoformat("T") + "Z"
march_end = datetime(current_date.year, 3, 31, 23, 59, 59).isoformat("T") + "Z"

Se vuelve a correr la consulta de el mes de enero.

In [107]:
videos = []
for _, row in noticias.iterrows():
    channel_id = row['ChannelID']
    categoria = row['Categoría']
    nombre_noticiero = row['Nombre del noticiero']
    
    if pd.notnull(channel_id):
        for _, clave_row in claves.iterrows():
            query = clave_row['Query']
            country = clave_row['País']
            search_results = search_videos(channel_id, query, march_first, march_end)
            for video in search_results:
                if video:
                    video["Nombre del noticiero"] = nombre_noticiero
                    video["Query"] = query  # Agregar la consulta a los datos del video
                    if categoria == "Internacional":
                        video["País"] = f"Internacional + {country}"
                    else:
                        video["País"] = categoria
                    videos.append(video)

# Crear DataFrame y guardar en CSV
df_videos = pd.DataFrame(videos)
df_videos.to_csv("./videos-per-month/marzo.csv", index=False)

print("Videos data saved to ./videos-per-month/marzo.csv")

Videos data saved to ./videos-per-month/marzo.csv


In [108]:
df_videos["Numero de Comentarios"].sum()

101775

## Extracción de datos de Abril

In [109]:
api_key = "your_api_key"

In [110]:
youtube = build('youtube', 'v3', developerKey=api_key)

In [111]:
current_date = datetime.now()
april_first = datetime(current_date.year, 4, 1).isoformat("T") + "Z"
april_end = datetime(current_date.year, 4, 30, 23, 59, 59).isoformat("T") + "Z"

Se vuelve a correr la consulta de el mes de enero con las fechas modificadas.

In [112]:
videos = []
for _, row in noticias.iterrows():
    channel_id = row['ChannelID']
    categoria = row['Categoría']
    nombre_noticiero = row['Nombre del noticiero']
    
    if pd.notnull(channel_id):
        for _, clave_row in claves.iterrows():
            query = clave_row['Query']
            country = clave_row['País']
            search_results = search_videos(channel_id, query, april_first, april_end)
            for video in search_results:
                if video:
                    video["Nombre del noticiero"] = nombre_noticiero
                    video["Query"] = query  # Agregar la consulta a los datos del video
                    if categoria == "Internacional":
                        video["País"] = f"Internacional + {country}"
                    else:
                        video["País"] = categoria
                    videos.append(video)

# Crear DataFrame y guardar en CSV
df_videos = pd.DataFrame(videos)
df_videos.to_csv("./videos-per-month/abril.csv", index=False)

print("Videos data saved to ./videos-per-month/abril.csv")

Videos data saved to ./videos-per-month/abril.csv


In [113]:
df_videos["Numero de Comentarios"].sum()

342488

## Extracción de datos de Mayo

In [134]:
api_key = "your_api_key"

In [35]:
youtube = build('youtube', 'v3', developerKey=api_key)

In [120]:
current_date = datetime.now()
may_first = datetime(current_date.year, 5, 1).isoformat("T") + "Z"
may_end = datetime(current_date.year, 5, 31, 23, 59, 59).isoformat("T") + "Z"

Se vuelve a correr la consulta de el mes de enero con las fechas modificadas.

In [121]:
videos = []
for _, row in noticias.iterrows():
    channel_id = row['ChannelID']
    categoria = row['Categoría']
    nombre_noticiero = row['Nombre del noticiero']
    
    if pd.notnull(channel_id):
        for _, clave_row in claves.iterrows():
            query = clave_row['Query']
            country = clave_row['País']
            search_results = search_videos(channel_id, query, may_first, may_end)
            for video in search_results:
                if video:
                    video["Nombre del noticiero"] = nombre_noticiero
                    video["Query"] = query  # Agregar la consulta a los datos del video
                    if categoria == "Internacional":
                        video["País"] = f"Internacional + {country}"
                    else:
                        video["País"] = categoria
                    videos.append(video)

# Crear DataFrame y guardar en CSV
df_videos = pd.DataFrame(videos)
df_videos.to_csv("./videos-per-month/mayo2.csv", index=False)

print("Videos data saved to ./videos-per-month/mayo.csv")

Videos data saved to ./videos-per-month/mayo.csv


In [122]:
df_videos["Numero de Comentarios"].sum()

118376

## Extracción de datos de Junio

In [26]:
api_key = "your_api_key"

In [27]:
youtube = build('youtube', 'v3', developerKey=api_key)

In [28]:
current_date = datetime.now()
june_first = datetime(current_date.year, 6, 1).isoformat("T") + "Z"
june_end = datetime(current_date.year, 6, 30, 23, 59, 59).isoformat("T") + "Z"

Se vuelve a correr la consulta de el mes de enero con las fechas modificadas.

In [29]:
videos = []
for _, row in noticias.iterrows():
    channel_id = row['ChannelID']
    categoria = row['Categoría']
    nombre_noticiero = row['Nombre del noticiero']
    
    if pd.notnull(channel_id):
        for _, clave_row in claves.iterrows():
            query = clave_row['Query']
            country = clave_row['País']
            search_results = search_videos(channel_id, query, june_first, june_end)
            for video in search_results:
                if video:
                    video["Nombre del noticiero"] = nombre_noticiero
                    video["Query"] = query  # Agregar la consulta a los datos del video
                    if categoria == "Internacional":
                        video["País"] = f"Internacional + {country}"
                    else:
                        video["País"] = categoria
                    videos.append(video)

# Crear DataFrame y guardar en CSV
df_videos = pd.DataFrame(videos)
df_videos.to_csv("./videos-per-month/junio.csv", index=False)

print("Videos data saved to ./videos-per-month/junio.csv")

Videos data saved to ./videos-per-month/junio.csv


junio = pd.rea

In [31]:
junio = pd.read_csv("videos-per-month/junio.csv")

In [32]:
juniocomments = junio["Numero de Comentarios"].sum()

juniocomments

92792

# General Analysis

In [38]:
import pandas as pd
import plotly.express as px

In [106]:
# Sumar el número de comentarios de todos los videos
# leer los archivos csv
enero = pd.read_csv("./videos-per-month/enero.csv")
febrero = pd.read_csv("./videos-per-month/febrero.csv")
marzo = pd.read_csv("./videos-per-month/marzo.csv")
abril = pd.read_csv("./videos-per-month/abril.csv")
mayo = pd.read_csv("./videos-per-month/mayo.csv")
junio = pd.read_csv("videos-per-month/junio.csv")

# Concatenar los DataFrames
anual = pd.concat([enero, febrero, marzo, abril, mayo, junio])

# sumar el número de comentarios
total_comments = anual["Numero de Comentarios"].sum()


Se une en un solo dataset todos los meses extraidos.

In [ ]:
completo = anual

In [109]:
completo["Numero de Comentarios"].sum() # completo = 

977035

In [118]:
# necesito la grfica segmentada por pais

# Crear una columna para diferenciar entre Internacional y Nacional

completo['Tipo'] = completo['País'].apply(lambda x: 'Internacional' if 'Internacional' in x else 'Nacional')

# Agrupar los datos por 'País' y 'Tipo', sumando el número de comentarios
grouped_data = completo.groupby(['País', 'Tipo'])['Numero de Comentarios'].sum().unstack().fillna(0).reset_index()

# Crear la gráfica de barras apiladas
fig = go.Figure(data=[
    go.Bar(name='Internacional', x=grouped_data['País'], y=grouped_data['Internacional'], marker_color='blue'),
    go.Bar(name='Nacional', x=grouped_data['País'], y=grouped_data['Nacional'], marker_color='red')
])

# Cambiar el diseño de la gráfica para apilar las barras

fig.update_layout(barmode='stack', title='Comparación del Número de Comentarios por País',
                    xaxis_title='País', yaxis_title='Número de Comentarios')

# Mostrar la
fig.show()

In [141]:
completo.to_csv("completo.csv", index=False) # se renomnro a com_anuales para evitar confusiones con el nombre del archivo

In [ ]:
completo 

In [139]:
completo["Numero de Comentarios"].sum()

977035

In [ ]:
completo

# Identificando Duplicados y presidentes relacionados

In [11]:
import pandas as pd

In [23]:
com_anuales = pd.read_csv("./com_anuales.csv")

In [13]:
com_anuales["Numero de Comentarios"].sum()

977035

In [14]:
# mostrar duplicados

com_anuales[com_anuales.duplicated(subset="Link del Video", keep=False)]

,Nombre del Video,Link del Video,Numero de Comentarios,Duración del Video,Fecha del Video,Nombre del noticiero,Query,País,Tipo
0,"Una región, dos posturas: en Davos, Milei y Pe...",https://www.youtube.com/watch?v=hUB9Bbmh58I,871,PT6M29S,2024-01-18T01:53:49Z,France 24 Español,Argentina Alberto Fernández presidente,Internacional + Argentina,Internacional
1,El modelo Bukele: represión popular que seduce...,https://www.youtube.com/watch?v=WRYg1D--nDw,813,PT27M8S,2024-01-29T13:27:16Z,France 24 Español,El Salvador Nayib Bukele presidente,Internacional + El Salvador,Internacional
3,"Una región, dos posturas: en Davos, Milei y Pe...",https://www.youtube.com/watch?v=hUB9Bbmh58I,871,PT6M29S,2024-01-18T01:53:49Z,France 24 Español,Colombia Gustavo Petro presidente,Internacional + Colombia,Internacional
5,"Presidente de Ecuador, Daniel Noboa, anuncia e...",https://www.youtube.com/watch?v=QlefwZUQUTg,1945,PT2M,2024-01-10T18:01:49Z,Telemundo,El Salvador Nayib Bukele presidente,Internacional + El Salvador,Internacional
6,Impresionante video del momento del fatídico d...,https://www.youtube.com/watch?v=813ggEX6rfE,584,PT11M44S,2024-01-13T05:34:46Z,Telemundo,Colombia Gustavo Petro presidente,Internacional + Colombia,Internacional
...,...,...,...,...,...,...,...,...,...
586,"""Ha sido una paliza"": Ciro Gómez Leyva sobre e...",https://www.youtube.com/watch?v=JS1OAY7dvVk,10716,PT9M7S,2024-06-03T06:13:38Z,Imagen Televisión,México Andrés Manuel López Obrador presidente,México,Nacional
587,La oposición en México: Irresponsable y con pe...,https://www.youtube.com/watch?v=VnCXNc3SfQQ,1933,PT3M52S,2024-06-05T13:52:11Z,Imagen Televisión,México Andrés Manuel López Obrador presidente,México,Nacional
589,"Asesinaron a la candidata de Morena en Celaya,...",https://www.youtube.com/watch?v=gHzQxfH2moY,3672,PT8M38S,2024-04-02T04:41:46Z,Imagen Televisión,México Andrés Manuel López Obrador presidente,México,Nacional
593,Xóchitl Gálvez presenta sus propuestas económi...,https://www.youtube.com/watch?v=XJx5O_2X9nE,3030,PT8M20S,2024-04-20T02:40:28Z,Excélsior TV,México Andrés Manuel López Obrador presidente,México,Nacional


In [16]:
# mostrar filas con el mismo link en general Y NOMBRE para identificar si son duplicados o no

com_anuales[com_anuales.duplicated(subset="Link del Video", keep=False) & com_anuales.duplicated(subset="Nombre del Video", keep=False)]

,Nombre del Video,Link del Video,Numero de Comentarios,Duración del Video,Fecha del Video,Nombre del noticiero,Query,País,Tipo
0,"Una región, dos posturas: en Davos, Milei y Pe...",https://www.youtube.com/watch?v=hUB9Bbmh58I,871,PT6M29S,2024-01-18T01:53:49Z,France 24 Español,Argentina Alberto Fernández presidente,Internacional + Argentina,Internacional
1,El modelo Bukele: represión popular que seduce...,https://www.youtube.com/watch?v=WRYg1D--nDw,813,PT27M8S,2024-01-29T13:27:16Z,France 24 Español,El Salvador Nayib Bukele presidente,Internacional + El Salvador,Internacional
3,"Una región, dos posturas: en Davos, Milei y Pe...",https://www.youtube.com/watch?v=hUB9Bbmh58I,871,PT6M29S,2024-01-18T01:53:49Z,France 24 Español,Colombia Gustavo Petro presidente,Internacional + Colombia,Internacional
5,"Presidente de Ecuador, Daniel Noboa, anuncia e...",https://www.youtube.com/watch?v=QlefwZUQUTg,1945,PT2M,2024-01-10T18:01:49Z,Telemundo,El Salvador Nayib Bukele presidente,Internacional + El Salvador,Internacional
6,Impresionante video del momento del fatídico d...,https://www.youtube.com/watch?v=813ggEX6rfE,584,PT11M44S,2024-01-13T05:34:46Z,Telemundo,Colombia Gustavo Petro presidente,Internacional + Colombia,Internacional
...,...,...,...,...,...,...,...,...,...
586,"""Ha sido una paliza"": Ciro Gómez Leyva sobre e...",https://www.youtube.com/watch?v=JS1OAY7dvVk,10716,PT9M7S,2024-06-03T06:13:38Z,Imagen Televisión,México Andrés Manuel López Obrador presidente,México,Nacional
587,La oposición en México: Irresponsable y con pe...,https://www.youtube.com/watch?v=VnCXNc3SfQQ,1933,PT3M52S,2024-06-05T13:52:11Z,Imagen Televisión,México Andrés Manuel López Obrador presidente,México,Nacional
589,"Asesinaron a la candidata de Morena en Celaya,...",https://www.youtube.com/watch?v=gHzQxfH2moY,3672,PT8M38S,2024-04-02T04:41:46Z,Imagen Televisión,México Andrés Manuel López Obrador presidente,México,Nacional
593,Xóchitl Gálvez presenta sus propuestas económi...,https://www.youtube.com/watch?v=XJx5O_2X9nE,3030,PT8M20S,2024-04-20T02:40:28Z,Excélsior TV,México Andrés Manuel López Obrador presidente,México,Nacional


In [9]:
# drop completos con el mismo link 
com_anuales = com_anuales.drop_duplicates(subset=['Link del Video'])

com_anuales["Numero de Comentarios"].sum()

666824

In [33]:
com_anuales.to_csv("com_anuales_nd.csv", index=False) # se almacena el archivo sin duplicados eb com_anuales_nd para la extraccion de comentarios

In [36]:
# graficar la distribucion de comentarios de com_anuales en el tiempo

import plotly.express as px

com_anuales["Fecha del Video"] = pd.to_datetime(com_anuales["Fecha del Video"])

fig = px.histogram(com_anuales, x="Fecha del Video", title="Distribución de Fechas de Videos en com_anuales")

fig.show()

/var/folders/vz/gb8kps891y19rfdfwycr8ybr0000gn/T/ipykernel_90002/3339448812.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [37]:
com_anuales

,Nombre del Video,Link del Video,Numero de Comentarios,Duración del Video,Fecha del Video,Nombre del noticiero,Query,País,Tipo
0,"Una región, dos posturas: en Davos, Milei y Pe...",https://www.youtube.com/watch?v=hUB9Bbmh58I,871,PT6M29S,2024-01-18 01:53:49+00:00,France 24 Español,Argentina Alberto Fernández presidente,Internacional + Argentina,Internacional
1,El modelo Bukele: represión popular que seduce...,https://www.youtube.com/watch?v=WRYg1D--nDw,813,PT27M8S,2024-01-29 13:27:16+00:00,France 24 Español,El Salvador Nayib Bukele presidente,Internacional + El Salvador,Internacional
2,Bukele ordenó desmantelar un monumento del fin...,https://www.youtube.com/watch?v=ELfbIwDjcDU,154,PT1M29S,2024-01-04 14:46:23+00:00,France 24 Español,El Salvador Nayib Bukele presidente,Internacional + El Salvador,Internacional
4,Maneras opuestas de ver la economía: Milei y P...,https://www.youtube.com/watch?v=Zvhk-jDO33Y,592,PT1M41S,2024-01-18 01:53:52+00:00,France 24 Español,Colombia Gustavo Petro presidente,Internacional + Colombia,Internacional
5,"Presidente de Ecuador, Daniel Noboa, anuncia e...",https://www.youtube.com/watch?v=QlefwZUQUTg,1945,PT2M,2024-01-10 18:01:49+00:00,Telemundo,El Salvador Nayib Bukele presidente,Internacional + El Salvador,Internacional
...,...,...,...,...,...,...,...,...,...
559,"""A Petro y otros les viene bien dinamitar el t...",https://www.youtube.com/watch?v=CzyEcdJZAls,172,PT38M20S,2024-07-05 15:00:09+00:00,El espectador,México Andrés Manuel López Obrador presidente,Colombia,Nacional
588,Las enseñanzas que nos dejó la elección de 202...,https://www.youtube.com/watch?v=4U1SI4NnNPU,837,PT1M1S,2024-06-04 05:14:06+00:00,Imagen Televisión,México Andrés Manuel López Obrador presidente,México,Nacional
590,¿Petro infiel? Presidente de Colombia es visto...,https://www.youtube.com/watch?v=shRcmPdEuCI,396,PT49S,2024-07-02 18:00:24+00:00,Excélsior TV,Colombia Gustavo Petro presidente,México,Nacional
591,"¿Quién es Linda Yepes, la presentadora trans q...",https://www.youtube.com/watch?v=3BXUttfIrok,260,PT1M24S,2024-07-03 03:00:02+00:00,Excélsior TV,Colombia Gustavo Petro presidente,México,Nacional


In [38]:
# quitar la palabra internacional y el simbolo + de los valores de la columna pais para estructurar la informacion de manera correcta

com_anuales["País"] = com_anuales["País"].str.replace("Internacional", "").str.replace("+", "").str.strip()

/var/folders/vz/gb8kps891y19rfdfwycr8ybr0000gn/T/ipykernel_90002/1093309040.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [40]:
com_anuales["País"].unique()

array(['Argentina', 'El Salvador', 'Colombia', 'México'], dtype=object)

In [6]:
com_anuales.groupby(["Tipo", "País"])["Numero de Comentarios"].sum()


Tipo           País       
Internacional  Argentina        3732
               Colombia        38260
               El Salvador     12147
               México           6243
Nacional       Argentina      143047
               Colombia        40414
               El Salvador       621
               México         422360
Name: Numero de Comentarios, dtype: int64

In [7]:
com_anuales["Numero de Comentarios"].sum()

666824

# Extracción de comentarios

In [ ]:
com_anuales.head(10)

In [9]:
com_anuales['ID del Video'] = com_anuales['Link del Video'].apply(lambda x: x.split('v=')[-1]) # se obtiene el ID del video de la URL de cada video

In [ ]:
com_anuales

In [11]:
from googleapiclient.discovery import build
import pandas as pd

In [12]:
api_key = "your_api_key"
youtube = build('youtube', 'v3', developerKey=api_key)

In [13]:
def get_video_comments(video_id):
    comments_data = []
    request = youtube.commentThreads().list(
        part='snippet,replies', # se obtienen los metadatos de los comentarios
        videoId=video_id, # se especifica el ID del video
        maxResults=100 # se obtienen 100 comentarios por solicitud
    )
    response = request.execute()

    while request is not None: # se recorren los comentarios de los videos y sus respuestas asociadas si las hay 
        for item in response['items']: # procesamiento de comentarios princiaples
            comment = item['snippet']['topLevelComment']['snippet'] # se extrae la información del comentario
            comment_data = {
                'video_id': video_id, # se almacen el ID del video
                'comment_id': item['id'], # se almacen el ID del comentario
                'text': comment['textDisplay'], # se almacena el texto del comentario
                'likes': comment['likeCount'], # se almacena el número de likes del comentario
                'type': 'principal' # se almacena el tipo de comentario
            }
            comments_data.append(comment_data)
            
            if 'replies' in item: # procesamiento de respuestas a comentarios
                for reply in item['replies']['comments']: # se recorren las respuestas a los comentarios
                    reply_data = {
                        'video_id': video_id, # se almacena el ID del video
                        'comment_id': reply['id'], # se almacena el ID del comentario
                        'text': reply['snippet']['textDisplay'], # se almacena el texto del comentario
                        'likes': reply['snippet']['likeCount'], # se almacena el número de likes del comentario
                        'type': 'reply' # se almacen el tipo de comentario
                    }
                    comments_data.append(reply_data)
        
        if 'nextPageToken' in response: # como los comentarios pueden ser muchos, se hace paginacion, para obtener los siguientes comentarios se usa el token de la siguiente pagina
            request = youtube.commentThreads().list( # se realiza la solicitud para obtener los siguientes comentarios
                part='snippet,replies', # se obtienen los metadatos de los comentarios
                videoId=video_id, # se especifica el ID del video
                pageToken=response['nextPageToken'], # se especifica el token de la siguiente página de comentarios a obtener 
                maxResults=100 # se obtienen 100 comentarios por solicitud
            )
            response = request.execute()
        else:
            request = None
    return comments_data


In [14]:
all_comments_df = pd.DataFrame(columns=['video_id', 'comment_id', 'text', 'likes', 'type']) # se crea un DataFrame vacío para almacenar los comentarios

# Iterar sobre cada ID de video y extraer los comentarios
for video_id in com_anuales['ID del Video']: # se recorren los IDs de los videos
    comments = get_video_comments(video_id) # se obtienen los comentarios del video
    video_comments_df = pd.DataFrame(comments) # se crea un DataFrame con los comentarios del video
    all_comments_df = pd.concat([all_comments_df, video_comments_df], ignore_index=True) # se concatenan los comentarios del video al DataFrame principal de comentarios

# Guardar el DataFrame en un archivo CSV
all_comments_df.to_csv('comentarios_yt', index=False) # se almacenan los comentarios en un archivo CSV

In [16]:
all_comments_df

,video_id,comment_id,text,likes,type
0,hUB9Bbmh58I,UgxzYxtFuT8DX5IahFR4AaABAg,"No hablo de socialismo, que falacias que sacan",0,principal
1,hUB9Bbmh58I,UgyMvKupk0keUaOWGeF4AaABAg,"Vé, una pregunta: ¿Cuando Petro habló de socia...",0,principal
2,hUB9Bbmh58I,Ugzn5x-8V0cPAng1oYF4AaABAg,no entiendo porque quieren seguir vendiendo la...,0,principal
3,hUB9Bbmh58I,UgxXBXoLabae2Lb_vlB4AaABAg,"Selor Lucumi,Me gustaría aclarar que la postur...",0,principal
4,hUB9Bbmh58I,UgyLqVixbjXvEfZXKSR4AaABAg,JAJAJAJA ESE PETRO ES UN PAYASO...,0,principal
...,...,...,...,...,...
588500,KEUH4oKd1N0,Ugz9n49-qrnTLL-dhBF4AaABAg.A5SPkX9Z_ZlA5UtNbnraqw,"Será, por el teléfono o por el djdjwiwowbe ja...",1,reply
588501,KEUH4oKd1N0,Ugw9VVDlOJ_lj91JsBx4AaABAg,"Realidad de muchos, cogen hombres ricos pero s...",7,principal
588502,KEUH4oKd1N0,UgyEZZElbqo7A-024Jl4AaABAg,Un montaje totalmente Porque esos medios se va...,0,principal
588503,KEUH4oKd1N0,Ugw_EE7cruqg1axy0hl4AaABAg,Montaje.,4,principal


In [1]:
import pandas as pd

In [5]:
comentarios = pd.read_csv("./data/complete/comentarios_videos.csv")

In [6]:
comentarios

,video_id,comment_id,Texto Comentario,likes,Tipo Comentaro,Nombre del Video,Link del Video,Numero de Comentarios,Duración del Video,Fecha del Video,Nombre del noticiero,Query,País,Tipo Noticiero,País_Medio,Duración en Minutos,Nombre del Presidente
0,hUB9Bbmh58I,UgxzYxtFuT8DX5IahFR4AaABAg,"No hablo de socialismo, que falacias que sacan",0.0,principal,"Una región, dos posturas: en Davos, Milei y Pe...",https://www.youtube.com/watch?v=hUB9Bbmh58I,871,PT6M29S,2024-01-18 01:53:49+00:00,France 24 Español,Argentina Javier Milei presidente,Argentina,Internacional,Internacional,6.483333,Javier Milei
1,hUB9Bbmh58I,UgyMvKupk0keUaOWGeF4AaABAg,"Vé, una pregunta: ¿Cuando Petro habló de socia...",0.0,principal,"Una región, dos posturas: en Davos, Milei y Pe...",https://www.youtube.com/watch?v=hUB9Bbmh58I,871,PT6M29S,2024-01-18 01:53:49+00:00,France 24 Español,Argentina Javier Milei presidente,Argentina,Internacional,Internacional,6.483333,Javier Milei
2,hUB9Bbmh58I,Ugzn5x-8V0cPAng1oYF4AaABAg,no entiendo porque quieren seguir vendiendo la...,0.0,principal,"Una región, dos posturas: en Davos, Milei y Pe...",https://www.youtube.com/watch?v=hUB9Bbmh58I,871,PT6M29S,2024-01-18 01:53:49+00:00,France 24 Español,Argentina Javier Milei presidente,Argentina,Internacional,Internacional,6.483333,Javier Milei
3,hUB9Bbmh58I,UgxXBXoLabae2Lb_vlB4AaABAg,"Selor Lucumi,Me gustaría aclarar que la postur...",0.0,principal,"Una región, dos posturas: en Davos, Milei y Pe...",https://www.youtube.com/watch?v=hUB9Bbmh58I,871,PT6M29S,2024-01-18 01:53:49+00:00,France 24 Español,Argentina Javier Milei presidente,Argentina,Internacional,Internacional,6.483333,Javier Milei
4,hUB9Bbmh58I,UgyLqVixbjXvEfZXKSR4AaABAg,JAJAJAJA ESE PETRO ES UN PAYASO...,0.0,principal,"Una región, dos posturas: en Davos, Milei y Pe...",https://www.youtube.com/watch?v=hUB9Bbmh58I,871,PT6M29S,2024-01-18 01:53:49+00:00,France 24 Español,Argentina Javier Milei presidente,Argentina,Internacional,Internacional,6.483333,Javier Milei
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
588500,KEUH4oKd1N0,Ugz9n49-qrnTLL-dhBF4AaABAg.A5SPkX9Z_ZlA5UtNbnraqw,"Será, por el teléfono o por el djdjwiwowbe ja...",1.0,reply,"Escándalo en Colombia: “Soy heterosexual”, dic...",https://www.youtube.com/watch?v=KEUH4oKd1N0,130,PT1M14S,2024-07-04 01:15:01+00:00,Excélsior TV,Colombia Gustavo Petro presidente,México,Nacional,México,1.233333,Andrés Manuel López Obrador
588501,KEUH4oKd1N0,Ugw9VVDlOJ_lj91JsBx4AaABAg,"Realidad de muchos, cogen hombres ricos pero s...",7.0,principal,"Escándalo en Colombia: “Soy heterosexual”, dic...",https://www.youtube.com/watch?v=KEUH4oKd1N0,130,PT1M14S,2024-07-04 01:15:01+00:00,Excélsior TV,Colombia Gustavo Petro presidente,México,Nacional,México,1.233333,Andrés Manuel López Obrador
588502,KEUH4oKd1N0,UgyEZZElbqo7A-024Jl4AaABAg,Un montaje totalmente Porque esos medios se va...,0.0,principal,"Escándalo en Colombia: “Soy heterosexual”, dic...",https://www.youtube.com/watch?v=KEUH4oKd1N0,130,PT1M14S,2024-07-04 01:15:01+00:00,Excélsior TV,Colombia Gustavo Petro presidente,México,Nacional,México,1.233333,Andrés Manuel López Obrador
588503,KEUH4oKd1N0,Ugw_EE7cruqg1axy0hl4AaABAg,Montaje.,4.0,principal,"Escándalo en Colombia: “Soy heterosexual”, dic...",https://www.youtube.com/watch?v=KEUH4oKd1N0,130,PT1M14S,2024-07-04 01:15:01+00:00,Excélsior TV,Colombia Gustavo Petro presidente,México,Nacional,México,1.233333,Andrés Manuel López Obrador
